In [18]:
from dash import Dash, html, dcc, dash_table, Input, Output, callback
import pandas as pd
import plotly.express as px

app = Dash(__name__)
default_table = pd.read_csv('../data/cleaned_fantasy_1970.csv')
app.layout = [html.Title('Fantasy Football Dashboard'),
              html.H1('Fantasy Football Dashboard', style={'textAlign': 'center', 'color': 'white'}),
              html.Div(children=[
                  dcc.Dropdown(id='year-dropdown', options=[{'label': i, 'value': i} for i in range(1970, 2024, 1)], value=1970),
                  dcc.Dropdown(id='position-dropdown', options=[{'label': i, 'value': i} for i in default_table['position'].unique()], value='All')
              ]),
              html.Div(children=[dash_table.DataTable(id='fantasy-football-table',data=default_table.to_dict('records'), page_size=10)]),
              html.Div(children=[
                dcc.Graph(id='fantasy-football-graph',figure={})])
            ]

@callback(
    Output('fantasy-football-graph', 'figure'),
    Output('fantasy-football-table', 'data'),
    Input('year-dropdown', 'value')
)
def update_table_and_graph(selected_year):
    df = pd.read_csv(f'../data/cleaned_fantasy_{selected_year}.csv')
    fig = px.pie(df, df['position'], title = f'Top 100 Fantasy Football Players by Position in {selected_year}')
    return fig, df.to_dict('records')

@callback(
    Output('fantasy-football-table', 'data'),
    Input('year-dropdown', 'value')
)
def update_table(selected_year, selected_position):
    df = pd.read_csv(f'../data/cleaned_fantasy_{selected_year}.csv')
    if selected_position == 'All':
        return df.to_dict('records')
    else:
        return df[df['position'] == selected_position].to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)